## Using Coach as a Library

Coach can be used as a library directly from python. As described in previous notebooks, Coach uses the presets mechanism to define the experiments. A preset is essentially a python module which instantiates a `GraphManager` object. The graph manager is a container that holds the agents and the environments, and has some additional parameters for running the experiment, such as visualization parameters. The graph manager acts as the scheduler which orchestrates the experiment.
Running Coach directly from python is done through a `CoachInterface` object, which uses the same arguments as the command line invocation but allowes for more flexibility and additional control of the training/inference process.

Let's start with some examples.

Creating a very simple graph containing a single Clipped PPO agent running with the CartPole-v0 Gym environment:


In [ ]:
# Adding module path to sys path if not there, so rl_coach submodules can be imported
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from rl_coach.coach import CoachInterface

In [ ]:
coach = CoachInterface(preset='CartPole_ClippedPPO',
                       custom_parameter='heatup_steps=EnvironmentSteps(5);improve_steps=TrainingSteps(3)')

Running the graph according to the given schedule:

In [ ]:
coach.run()

### Running each phase manually

The graph manager (which was instantiated in the preset) can be accessed from the `CoachInterface` object. The graph manager simplifies the scheduling process by encapsulating the calls to each of the training phases. Sometimes, it can be beneficial to have a more fine grained control over the scheduling process. This can be easily done by calling the individual phase functions directly:

In [ ]:
from rl_coach.core_types import EnvironmentSteps

coach.graph_manager.heatup(EnvironmentSteps(100))
for _ in range(10):
    coach.graph_manager.train_and_act(EnvironmentSteps(50))

### Additional functionality

`CoachInterface` allows for easy access to functionalities such as multi-threading and saving checkpoints:

In [ ]:
coach = CoachInterface(preset='CartPole_ClippedPPO', num_workers=2, checkpoint_save_secs=10)
coach.run()

### Agent functionality

When using `CoachInterface` (single agent with one level of hierarchy) it's also possible to easily use the `Agent` object functionality, such as logging signals and applying the policy the agent has learned on a given state:

In [ ]:
from rl_coach.environments.gym_environment import GymEnvironment,  GymVectorEnvironment
from rl_coach.base_parameters import VisualizationParameters
from rl_coach.core_types import EnvironmentSteps

coach = CoachInterface(preset='CartPole_ClippedPPO')

# training
for it in range(10):
    coach.graph_manager.train_and_act(EnvironmentSteps(100))
    coach.graph_manager.log_signal('iteration', it)

In [ ]:
# inference
env_params = GymVectorEnvironment(level='CartPole-v0')
env = GymEnvironment(**env_params.__dict__, visualization_parameters=VisualizationParameters())

for it in range(10):
    action_info = coach.graph_manager.get_agent().choose_action(env.state)
    print("State:{}, Action:{}".format(env.state,action_info.action))
    env.step(action_info.action)